# python-MIP

In [ ]:
!pip install mip

In [ ]:
from mip import *
from sys import stdout

## knapsack

In [ ]:
p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {}".format(selected))

selected items: [0, 3]


## n-queens

In [ ]:
# number of queens
n = 10

queens = Model()

x = [[queens.add_var('x({},{})'.format(i, j), var_type=BINARY)
      for j in range(n)] for i in range(n)]

# one per row
for i in range(n):
    queens += xsum(x[i][j] for j in range(n)) == 1, 'row({})'.format(i)

# one per column
for j in range(n):
    queens += xsum(x[i][j] for i in range(n)) == 1, 'col({})'.format(j)

# diagonal \
for p, k in enumerate(range(2 - n, n - 2 + 1)):
    queens += xsum(x[i][i - k] for i in range(n)
                   if 0 <= i - k < n) <= 1, 'diag1({})'.format(p)

# diagonal /
for p, k in enumerate(range(3, n + n)):
    queens += xsum(x[i][k - i] for i in range(n)
                   if 0 <= k - i < n) <= 1, 'diag2({})'.format(p)

queens.optimize()

if queens.num_solutions:
    stdout.write('\n')
    for i, v in enumerate(queens.vars):
        stdout.write('O ' if v.x >= 0.99 else '. ')
        if i % n == n-1:
            stdout.write('\n')


. . . . . . . O . . 
. . O . . . . . . . 
. . . . O . . . . . 
. . . . . . . . . O 
. O . . . . . . . . 
. . . . . . . . O . 
. . . . . O . . . . 
. . . O . . . . . . 
. . . . . . O . . . 
O . . . . . . . . . 


## sport tournament

In [ ]:
n = 4

srrt = Model()

x = [[[[srrt.add_var('x({},{},{},{})'.format(i, j, p, w), var_type=BINARY)
     for w in range(n-1)] for p in range(int(n/2))] for j in range(n)] for i in range(n)]


# one per row
for i in range(n):
    for p in range(int(n/2)):
        for w in range(n-1):
            srrt += xsum(x[i][j][p][w] for j in range(n)) == 1, 'row({})'.format(i)

# one per column
for j in range(n):
    for p in range(int(n/2)):
        for w in range(n-1):
            srrt += xsum(x[i][j][p][w] for i in range(n)) == 1, 'col({})'.format(j)

# srrt.objective = minimize(xsum(x[i][j][p][w] for w in range(n-1) for p in range(int(n/2)) for j in range(n) for i in range(n)))

status = srrt.optimize(max_seconds=300)
print(status)

print(srrt.num_solutions)

print(srrt.vars)
for i, v in enumerate(srrt.vars):
    stdout.write('O ' if v.x >= 0.99 else '. ')
    if i % n == n-1:
        stdout.write('\n')
    if i % n**2 == (n**2-1):
        stdout.write('\n')

OptimizationStatus.OPTIMAL
1
. . . . 
. . . . 
. . . O 
. . . . 

. . O O 
O O O . 
. . . . 
. . O O 

. . . . 
. . O O 
O O . . 
. . . . 

. . . . 
. . . . 
O O O . 
O O . . 

. . . . 
. . . O 
O O O O 
O O . . 

. . . . 
. . . . 
. . . . 
. . . . 



# PuLP

solver independent

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 40.0 MB/s eta 0:00:00


In [ ]:
import pulp as pl
import numpy as np

## sudoku

In [ ]:
VALS = ROWS = COLS = range(1, 10)
Boxes = [
    [(3 * i + k + 1, 3 * j + l + 1) for k in range(3) for l in range(3)]
    for i in range(3)
    for j in range(3)
]
prob = LpProblem("Sudoku Problem")
choices = LpVariable.dicts("Choice", (VALS, ROWS, COLS), cat="Binary")
for r in ROWS:
    for c in COLS:
        prob += lpSum([choices[v][r][c] for v in VALS]) == 1

# The row, column and box constraints are added for each value
for v in VALS:
    for r in ROWS:
        prob += lpSum([choices[v][r][c] for c in COLS]) == 1

    for c in COLS:
        prob += lpSum([choices[v][r][c] for r in ROWS]) == 1

    for b in Boxes:
        prob += lpSum([choices[v][r][c] for (r, c) in b]) == 1

# The starting numbers are entered as constraints
input_data = [
    (5, 1, 1),
    (6, 2, 1),
    (8, 4, 1),
    (4, 5, 1),
    (7, 6, 1),
    (3, 1, 2),
    (9, 3, 2),
    (6, 7, 2),
    (8, 3, 3),
    (1, 2, 4),
    (8, 5, 4),
    (4, 8, 4),
    (7, 1, 5),
    (9, 2, 5),
    (6, 4, 5),
    (2, 6, 5),
    (1, 8, 5),
    (8, 9, 5),
    (5, 2, 6),
    (3, 5, 6),
    (9, 8, 6),
    (2, 7, 7),
    (6, 3, 8),
    (8, 7, 8),
    (7, 9, 8),
    (3, 4, 9),
    (1, 5, 9),
    (6, 6, 9),
    (5, 8, 9),
]

for v, r, c in input_data:
    prob += choices[v][r][c] == 1

# The problem data is written to an .lp file
prob.writeLP("Sudoku.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# A file called sudokuout.txt is created/overwritten for writing to
sudokuout = open("sudokuout.txt", "w")

# The solution is written to the sudokuout.txt file
for r in ROWS:
    if r in [1, 4, 7]:
        sudokuout.write("+-------+-------+-------+\n")
    for c in COLS:
        for v in VALS:
            if value(choices[v][r][c]) == 1:
                if c in [1, 4, 7]:
                    sudokuout.write("| ")
                sudokuout.write(str(v) + " ")
                if c == 9:
                    sudokuout.write("|\n")
sudokuout.write("+-------+-------+-------+")
sudokuout.close()

Status: Optimal
Status: Optimal
Status: Infeasible


In [ ]:
sudokuout = open("sudokuout.txt", "w")
while True:
    prob.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[prob.status])
    # The solution is printed if it was deemed "optimal" i.e met the constraints
    if LpStatus[prob.status] == "Optimal":
        # The solution is written to the sudokuout.txt file
        for r in ROWS:
            if r in [1, 4, 7]:
                sudokuout.write("+-------+-------+-------+\n")
            for c in COLS:
                for v in VALS:
                    if value(choices[v][r][c]) == 1:
                        if c in [1, 4, 7]:
                            sudokuout.write("| ")
                        sudokuout.write(str(v) + " ")
                        if c == 9:
                            sudokuout.write("|\n")
        sudokuout.write("+-------+-------+-------+\n\n")
        # The constraint is added that the same solution cannot be returned again
        prob += (
            lpSum(
                [
                    choices[v][r][c]
                    for v in VALS
                    for r in ROWS
                    for c in COLS
                    if value(choices[v][r][c]) == 1
                ]
            )
            <= 80
        )
    # If a new optimal solution cannot be found, we end the program
    else:
        break
sudokuout.close()

Status: Optimal
Status: Infeasible


## RR Tournament Scheduling

In [ ]:
# Parametri
n = 8
weeks = n - 1
periods = n // 2

# Modello
model = pl.LpProblem("RR_Tournament_Scheduling", pl.LpMinimize)

# Variabili
x = pl.LpVariable.dicts("x", ((w, p, i, j) for w in range(weeks) for p in range(periods) for i in range(n) for j in range(n)),
                        cat=pl.LpBinary)

h = pl.LpVariable.dicts("home", (i for i in range(n)), cat=pl.LpInteger)
a = pl.LpVariable.dicts("away", (i for i in range(n)), cat=pl.LpInteger)
d = pl.LpVariable.dicts("diff", (i for i in range(n)), lowBound=0, cat=pl.LpContinuous)

# Vincoli

# Ogni squadra gioca contro tutte le altre esattamente una volta
for i in range(n):
    model += (
        pl.lpSum(x[w, p, i, j] for w in range(weeks) for p in range(periods) for j in range(n)) +
        pl.lpSum(x[w, p, j, i] for w in range(weeks) for p in range(periods) for j in range(n))
        == n - 1
    ), f"MatchCount_{i}"

# Nessuna squadra gioca contro sé stessa
for i in range(n):
    for w in range(weeks):
        for p in range(periods):
            model += x[w, p, i, i] == 0, f"NoSelfMatch_w{w}_p{p}_i{i}"

# Ogni coppia di squadre gioca una sola volta
for i in range(n):
    for j in range(n):
        if i != j:
            model += (
                pl.lpSum(x[w, p, i, j] for w in range(weeks) for p in range(periods)) +
                pl.lpSum(x[w, p, j, i] for w in range(weeks) for p in range(periods))
                == 1
            ), f"OneMatchPerPair_{i}_{j}"

# Una sola partita per squadra per settimana
for w in range(weeks):
    for i in range(n):
        model += pl.lpSum(x[w, p, i, j] for p in range(periods) for j in range(n)) <= 1, f"OneMatchPerWeek_i{i}_w{w}"
    for j in range(n):
        model += pl.lpSum(x[w, p, i, j] for p in range(periods) for i in range(n)) <= 1, f"OneMatchPerWeek_j{j}_w{w}"

# Al massimo una partita per periodo per settimana
for w in range(weeks):
    for i in range(n):
        for j in range(n):
            model += pl.lpSum(x[w, p, i, j] for p in range(periods)) <= 1, f"MaxOneMatchPerPeriod_w{w}_i{i}_j{j}"

# Al massimo una partita per ogni combinazione i-j
for i in range(n):
    for j in range(n):
        model += pl.lpSum(x[w, p, i, j] for w in range(weeks) for p in range(periods)) <= 1, f"MaxOneGamePerPair_{i}_{j}"

# Al massimo 2 partite per squadra per periodo su tutte le settimane
for i in range(n):
    for p in range(periods):
        model += (
            pl.lpSum(x[w, p, i, j] for w in range(weeks) for j in range(n)) +
            pl.lpSum(x[w, p, j, i] for w in range(weeks) for j in range(n))
            <= 2
        ), f"MaxTwoGamesPerPeriod_{i}_p{p}"

# Definizione home, away, diff
for i in range(n):
    model += h[i] == pl.lpSum(x[w, p, i, j] for w in range(weeks) for p in range(periods) for j in range(n) if i != j), f"HomeCount_{i}"
    model += a[i] == pl.lpSum(x[w, p, j, i] for w in range(weeks) for p in range(periods) for j in range(n) if i != j), f"AwayCount_{i}"
    model += d[i] >= h[i] - a[i], f"DiffPos_{i}"
    model += d[i] >= a[i] - h[i], f"DiffNeg_{i}"

# Funzione obiettivo
model += pl.lpSum(d[i] for i in range(n)), "MinimizeTotalDifference"

# Risoluzione
solver = pl.PULP_CBC_CMD(msg=1, timeLimit=300)
model.solve(solver)

# Output soluzione
solution = {}
for w in range(weeks):
    for p in range(periods):
        for i in range(n):
            for j in range(n):
                if pl.value(x[w, p, i, j]) > 0.5:
                    solution[(w, p, i, j)] = 1

In [ ]:
x_matrix = np.zeros((weeks, periods, n, n))

for w in range(weeks):
    for p in range(periods):
        for i in range(n):
            for j in range(n):
                x_matrix[w, p, i, j] = pl.value(x[w, p, i, j])

print("Objective val: ", model.objective.value())
print("coincise matrix: \n", np.sum(x_matrix, axis=(0, 1)))
# print("x:", x_matrix)

Objective val:  8.0
coincise matrix: 
 [[0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 0. 0.]]


# gurobi

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 46.4 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from datetime import datetime

## es.

In [ ]:
env = gp.Env()

model = gp.Model(env=env)
x = model.addVar(vtype=GRB.BINARY, name="x")
y = model.addVar(vtype=GRB.BINARY, name="y")
z = model.addVar(vtype=GRB.BINARY, name="z")

model.setObjective(x + y + 2 * z, GRB.MAXIMIZE)
model.addConstr(x + 2 * y + 3 * z <= 4, "c0")

model.Params.TimeLimit = 10.0
model.optimize()

print(f"x = {x.X}")
print(f"y = {y.X}")
print(f"z = {z.X}")

Restricted license - for non-production use only - expires 2026-11-23
Set parameter TimeLimit to value 10
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  10

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x3eaaf627
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance

## sudoku

In [ ]:
n = 8

m = gp.Model("nqueens")

# n-by-n binary variables; x[i, j] decides whether a queen is placed at
# position (i, j)
x = m.addMVar((n, n), vtype=GRB.BINARY, name="x")

# Maximize the number of placed queens
m.setObjective(x.sum(), GRB.MAXIMIZE)

# At most one queen per row; this adds n linear constraints
m.addConstr(x.sum(axis=1) <= 1, name="row")

# At most one queen per column; this adds n linear constraints
m.addConstr(x.sum(axis=0) <= 1, name="col")

for i in range(-n + 1, n):
    # At most one queen on diagonal i
    m.addConstr(x.diagonal(i).sum() <= 1, name=f"diag{i:d}")

    # At most one queen on anti-diagonal i
    m.addConstr(x[:, ::-1].diagonal(i).sum() <= 1, name=f"adiag{i:d}")

# Solve the problem
m.optimize()

print(x.X)
print(f"Queens placed: {m.ObjVal:.0f}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 46 rows, 64 columns and 256 nonzeros
Model fingerprint: 0xf9127510
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 42 rows, 64 columns, 270 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)

Root relaxation: objective 8.000000e+00, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0        

## RR tournament scheduling

In [ ]:
n = 5
m = gp.Model("RR Tournament Scheduling")

### contraints

#### constraint 1

In [ ]:
x = m.addMVar((n-1, n//2, n, n), vtype=GRB.BINARY, name="x")

# one match against each team
m.addConstrs((
    gp.quicksum(x[i1, i2, i, i4] for i1 in range(n-1) for i2 in range(n//2) for i4 in range(n)) +
    gp.quicksum(x[i1, i2, i3, i] for i1 in range(n-1) for i2 in range(n//2) for i3 in range(n))
    == n - 1
    for i in range(n)
), name="ij")

# no match against itself
m.addConstrs((x[:, :, i, i] == 0 for i in range(n)), name="self")

# diag symmetry breaking
m.addConstrs((
    gp.quicksum(x[i1, i2, i, j] for i1 in range(n-1) for i2 in range(n//2)) +
    gp.quicksum(x[i1, i2, j, i] for i1 in range(n-1) for i2 in range(n//2))
    == 1
    for i in range(n) for j in range(n) if i != j
), name="ij")

# one match per team per week
for w in range(n-1):
    m.addConstrs((x[w, :, i, :].sum() <= 1 for i in range(n)), name="max_w")
    m.addConstrs((x[w, :, :, j].sum() <= 1 for j in range(n)), name="max_w")

# max one match over the periods in a week
for w in range(n-1):
    m.addConstrs((x[w, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_p")

# max one game per match
m.addConstrs((x[:, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_ij")

# at most 2 match per period per team
m.addConstrs((
    gp.quicksum(x[w, p, i, j] for w in range(n-1) for j in range(n)) +
    gp.quicksum(x[w, p, j, i] for w in range(n-1) for j in range(n))
    <= 2
    for i in range(n) for p in range(n//2)
), name="ij")


# Objective function
h = m.addMVar((n), vtype=GRB.INTEGER, name="home")
a = m.addMVar((n), vtype=GRB.INTEGER, name="away")
d = m.addMVar((n), vtype=GRB.CONTINUOUS, name="diff")

m.addConstrs((h[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((a[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((d[i] >= h[i] - a[i] for i in range(n)))
m.addConstrs((d[i] >= a[i] - h[i] for i in range(n)))

m.setObjective(d.sum(), GRB.MINIMIZE)

#### contraint 2

In [ ]:
#################### by gippy

x = m.addMVar((n-1, n//2, n, n), vtype=GRB.BINARY, name="x")

for i in range(n):
    m.addConstr(gp.quicksum(x[w, p, i, j] + x[w, p, j, i] for w in range(n - 1) for p in range(n // 2) for j in range(n) if j != i) == n - 1,
        name=f"games_per_team_{i}")
m.addConstrs((x[w, p, i, i] == 0 for w in range(n - 1) for p in range(n // 2) for i in range(n)),
    name="no_self_match")
for i in range(n):
    for j in range(i + 1, n):
        m.addConstr(gp.quicksum(x[w, p, i, j] + x[w, p, j, i] for w in range(n - 1) for p in range(n // 2)) == 1,
            name=f"unique_match_{i}_{j}")
for w in range(n - 1):
    for i in range(n):
        m.addConstr(gp.quicksum(x[w, p, i, j] + x[w, p, j, i] for p in range(n // 2) for j in range(n) if j != i) <= 1,
            name=f"one_match_per_week_team_{i}_week_{w}")
for w in range(n - 1):
    for i in range(n):
        for j in range(n):
            m.addConstr(gp.quicksum(x[w, p, i, j] for p in range(n // 2)) <= 1,
                name=f"one_match_pair_{i}_{j}_week_{w}")
for i in range(n):
    for j in range(n):
        m.addConstr(gp.quicksum(x[w, p, i, j] for w in range(n - 1) for p in range(n // 2)) <= 1,
            name=f"max_one_total_match_{i}_{j}")
for i in range(n):
    for p in range(n // 2):
        m.addConstr(gp.quicksum(x[w, p, i, j] + x[w, p, j, i] for w in range(n - 1) for j in range(n) if j != i) <= 2,
            name=f"max_2_matches_period_{p}_team_{i}")


# Objective function
h = m.addMVar((n), vtype=GRB.INTEGER, name="home")
a = m.addMVar((n), vtype=GRB.INTEGER, name="away")
d = m.addMVar((n), vtype=GRB.CONTINUOUS, name="diff")

m.addConstrs((h[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((a[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((d[i] >= h[i] - a[i] for i in range(n)))
m.addConstrs((d[i] >= a[i] - h[i] for i in range(n)))

m.setObjective(d.sum(), GRB.MINIMIZE)

### solution

In [ ]:
dt1 = datetime.now()
ts1 = datetime.timestamp(dt1)

m.Params.TimeLimit = 300.0
m.Params.PoolSearchMode = 2
m.Params.PoolSolutions = 5
m.optimize()
m.write("model.rlp")

dt2 = datetime.now()
ts2 = datetime.timestamp(dt2)
print(f"Time elapsed: {ts2 - ts1} seconds")

Set parameter TimeLimit to value 300
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 5
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  300
PoolSolutions  5
PoolSearchMode  2

Optimize a model with 260 rows, 215 columns and 2240 nonzeros
Model fingerprint: 0x891645bf
Variable types: 5 continuous, 210 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 200 rows and 50 columns
Presolve time: 0.00s
Presolved: 60 rows, 165 columns, 800 nonzeros
Variable types: 5 continuous, 160 integer (160 binary)

Root relaxation: objective 0.000000e+00, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    | 

In [ ]:
print(x.X)

[[[[ 0.  1. -0.  0.  0.]
   [-0.  0.  0. -0.  1.]
   [-0. -0.  0.  1.  0.]
   [ 1. -0. -0.  0.  0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0.  0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [ 1. -0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0.  1.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0.  0.  0.]
   [-0.  0.  1. -0. -0.]
   [ 1. -0.  0. -0. -0.]
   [-0.  1. -0.  0. -0.]
   [-

In [ ]:
m.Params.SolutionNumber = 0

print("N. of constr: ", m.NumConstrs)

# obtain the optimal value
print("Objective val: ", m.ObjVal)

# compress the solution in one array for test purposes
k = x.Xn.copy()
o = np.sum(k, axis=(0, 1))
print(o)

# print of the 3rd constr
for i in range(n):
    print(f"Team {i}:")
    for p in range(n//2):
        print(f"  Periodo {p}: {np.sum(k[:, p, i, :])+np.sum(k[:, p, :, i])} partite")

N. of constr:  260
Objective val:  0.0
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 1.]
 [1. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0.]]
Team 0:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
Team 1:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
Team 2:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
Team 3:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite
Team 4:
  Periodo 0: 2.0 partite
  Periodo 1: 2.0 partite


In [ ]:
# print all solutions found
for i in range(m.SolCount):
    m.Params.SolutionNumber = i
    print('Solution number: ', i)
    print("Objective val: ", m.PoolObjVal)

    # compress the solution in one array for test purposes
    o = np.sum(k, axis=(0, 1))
    print(o)
    print(np.reshape(x.Xn, (n-1, int(n/2), n, n)))
    print("")


Solution number:  0
Objective val:  0.0
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 1.]
 [1. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0.]]
[[[[ 0.  1. -0.  0.  0.]
   [-0.  0.  0. -0.  1.]
   [-0. -0.  0.  1.  0.]
   [ 1. -0. -0.  0.  0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0.  0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [ 1. -0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0.  1.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]

  [[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
   [-0. -0. -0. -0.  0.]]]


 [[[ 0. -0. -0. -0. -0.]
   [-0.  0. -0. -0. -0.]
   [-0. -0.  0. -0. -0.]
   [-0. -0. -0.  0. -0.]
 

# Pyomo

solvre independent

In [1]:
!pip install pyomo

In [2]:
from pyomo.environ import *
import pyomo.environ as pyo

solver installation

In [6]:
!apt-get install -y -qq glpk-utils
solver_path='/usr/bin/glpsol'

# !apt-get install -y -qq coinor-cbc
# solver_path='/usr/bin/cbc'

# !pip install gurobipy

## N-queens

In [ ]:
def solve_n_queens(n):
    model = ConcreteModel()

    # Indici delle celle
    model.I = RangeSet(0, n-1)
    model.J = RangeSet(0, n-1)
    model.X = Var(model.I, model.J, domain=Binary)

    # Vincoli: una regina per riga
    def row_rule(model, i):
        return sum(model.X[i, j] for j in model.J) == 1
    model.row_constraint = Constraint(model.I, rule=row_rule)

    # Vincoli: una regina per colonna
    def col_rule(model, j):
        return sum(model.X[i, j] for i in model.I) == 1
    model.col_constraint = Constraint(model.J, rule=col_rule)

    # Vincoli: diagonali ↘ (i - j costante)
    diagonals1 = set(i - j for i in range(n) for j in range(n))
    def diag1_rule(model, d):
        return sum(model.X[i, j] for i in range(n) for j in range(n) if i - j == d) <= 1
    model.diag1_constraint = Constraint(diagonals1, rule=diag1_rule)

    # Vincoli: diagonali ↖ (i + j costante)
    diagonals2 = set(i + j for i in range(n) for j in range(n))
    def diag2_rule(model, d):
        return sum(model.X[i, j] for i in range(n) for j in range(n) if i + j == d) <= 1
    model.diag2_constraint = Constraint(diagonals2, rule=diag2_rule)

    # Obiettivo fittizio (non serve massimizzare nulla, ma Pyomo richiede un obiettivo)
    model.obj = Objective(expr=1, sense=minimize)

    # Risolvi (usa CBC o altro solver installato)
    solver = SolverFactory('glpk', executable=solver_path)  # puoi usare anche 'glpk', 'gurobi', ecc.
    result = solver.solve(model, tee=False)

    # Stampa la soluzione
    board = [['.' for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if pyo.value(model.X[i, j]) > 0.5:
                board[i][j] = 'Q'

    # Visualizza la scacchiera
    for row in board:
        print(' '.join(row))

In [ ]:
solve_n_queens(16)

. . Q . . . . . . . . . . . . .
Q . . . . . . . . . . . . . . .
. . . . . Q . . . . . . . . . .
. . . . . . . Q . . . . . . . .
. . . . . . . . . Q . . . . . .
. . . . . . . . . . . . . . . Q
. . . . . . . . . . . . . Q . .
. . . . . . . . Q . . . . . . .
. . . . . . . . . . . . . . Q .
. . . Q . . . . . . . . . . . .
. Q . . . . . . . . . . . . . .
. . . . . . Q . . . . . . . . .
. . . . . . . . . . . . Q . . .
. . . . . . . . . . Q . . . . .
. . . . Q . . . . . . . . . . .
. . . . . . . . . . . Q . . . .


## RR tournament scheduling

#### homemade

In [ ]:
n = 4

# model definition
model = ConcreteModel()

# variable definition
model.W = RangeSet(0, (n-1)-1)
model.P = RangeSet(0, (n//2)-1)
model.I = RangeSet(0, n-1)
model.J = RangeSet(0, n-1)
model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)

# constraints
def match_per_team(model, i):
    return (sum(model.X[w,p,i,j] for w in model.W for p in model.P for j in model.J) +
            sum(model.X[w,p,j,i] for w in model.W for p in model.P for j in model.J)) == n-1
model.one_match_per_team = Constraint(model.I, rule=match_per_team)

def no_self(model, i):
    return sum(model.X[w,p,i,i] for w in model.W for p in model.P) == 0
model.no_self = Constraint(model.I, rule=no_self)

def diag_symmetry_breaking(model, i, j):
    if i != j:
        return (sum(model.X[w,p,i,j] for w in model.W for p in model.P) + sum(model.X[w,p,j,i] for w in model.W for p in model.P)) == 1
    else:
        return Constraint.Skip
model.diag_symmetry_breaking = Constraint(model.I, model.J, rule=diag_symmetry_breaking)

def one_match_per_team_per_week1(model, i):
    return sum(model.X[w,p,i,j] for w in model.W for p in model.P for j in model.J) <= 1
def one_match_per_team_per_week2(model, j):
    return sum(model.X[w,p,i,j] for w in model.W for p in model.P for i in model.I) <= 1
model.one_match_per_team_per_week1 = Constraint(model.I, rule=one_match_per_team_per_week1)
model.one_match_per_team_per_week2 = Constraint(model.J, rule=one_match_per_team_per_week2)

def one_match_per_period_per_week(model, i, j):
    return sum(model.X[w,p,j,i] for w in model.W for p in model.P) <= 1
model.one_match_per_period_per_week = Constraint(model.I, model.J, rule=one_match_per_period_per_week)

def one_game_per_match(model, i, j):
    return sum(model.X[w,p,i,j] for w in model.W for p in model.P) <= 1
model.one_game_per_match = Constraint(model.I, model.J, rule=one_game_per_match)

def max_2_match_per_period_per_team(model, p, i):
    return sum(model.X[w,p,i,j] for w in model.W for j in model.J) + sum(model.X[w,p,j,i] for w in model.W for j in model.J) <= 2
model.max_2_match_per_period_per_team = Constraint(model.P, model.I, rule=max_2_match_per_period_per_team)


# objective function
model.obj = Objective(expr=1, sense=minimize)

# solve
solver = SolverFactory('glpk', executable=solver_path)
result = solver.solve(model, tee=False)

# print
# for index in model.X:
#     var = model.X[index]
#     if var.value is not None and abs(var.value) > 1e-6:
#         print(f"X{index} = {var.value}")

print("result:", result)
# print("solution:")
# print(model.X.get_values())
for w in model.W:
    for p in model.P:
        matrix = []
        for i in model.I:
            row = []
            for j in model.J:
                var = model.X[w,p,i,j]
                if var.value is not None and abs(var.value) > 1e-6:
                    val = int(round(value(model.X[w, p, i, j])))
                else:
                    val = 0
                row.append(val)
            matrix.append(row)
        if sum(sum(i) for i in matrix) > 0:
            print(f"\nMatrix for w={w}, p={p}:")
            for row in matrix:
                print(row)

result: 
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 68
  Number of variables: 97
  Number of nonzeros: 888
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005818605422973633



#### gippy

In [20]:
n = 4

# model definition
model = ConcreteModel()

model.W = RangeSet(0, (n-1)-1)
model.P = RangeSet(0, (n//2)-1)
model.I = RangeSet(0, n-1)
model.J = RangeSet(0, n-1)

# Binary decision variable
model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)

# auxiliary vars for obj func
model.home = Var(model.I, domain=NonNegativeIntegers)
model.away = Var(model.I, domain=NonNegativeIntegers)
model.diff = Var(model.I, domain=NonNegativeReals)

# constraints
def one_match_rule(model, i):
    return (
        sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J) +
        sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J)
        == n - 1
    )
model.one_match_per_team = Constraint(model.I, rule=one_match_rule)

def no_self_match_rule(model, i):
    return sum(model.X[w, p, i, i] for w in model.W for p in model.P) == 0
model.no_self_match = Constraint(model.I, rule=no_self_match_rule)

def symmetry_rule(model, i, j):
    if i < j:
        return (
            sum(model.X[w, p, i, j] for w in model.W for p in model.P) +
            sum(model.X[w, p, j, i] for w in model.W for p in model.P)
            == 1
        )
    return Constraint.Skip
model.symmetry = Constraint(model.I, model.J, rule=symmetry_rule)

def one_match_per_week_rule(model, w, i):
    return sum(model.X[w, p, i, j] + model.X[w, p, j, i] for p in model.P for j in model.J if j != i) <= 1
model.max_one_match = Constraint(model.W, model.I, rule=one_match_per_week_rule)

def max_one_per_period_per_week(model, w, i, j):
    return sum(model.X[w, p, i, j] for p in model.P) <= 1
model.max_one_period = Constraint(model.W, model.I, model.J, rule=max_one_per_period_per_week)

def max_one_game_per_match(model, i, j):
    return sum(model.X[w, p, i, j] for w in model.W for p in model.P) <= 1
model.max_one_match = Constraint(model.I, model.J, rule=max_one_game_per_match)

def max_one_game_per_match(model, i, j):
    return sum(model.X[w, p, i, j] for w in model.W for p in model.P) <= 1
model.max_one_match = Constraint(model.I, model.J, rule=max_one_game_per_match)

def max_two_matches_per_period(model, i, p):
    return (
        sum(model.X[w, p, i, j] for w in model.W for j in model.J) +
        sum(model.X[w, p, j, i] for w in model.W for j in model.J)
        <= 2
    )
model.max_two_matches = Constraint(model.I, model.P, rule=max_two_matches_per_period)

# auxiliary constraints for obj function
def home_games_rule(model, i):
    return model.home[i] == sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J if i != j)
model.home_games = Constraint(model.I, rule=home_games_rule)

def away_games_rule(model, i):
    return model.away[i] == sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J if i != j)
model.away_games = Constraint(model.I, rule=away_games_rule)

def diff_lower_bound1(model, i):
    return model.diff[i] >= model.home[i] - model.away[i]
model.diff_lb1 = Constraint(model.I, rule=diff_lower_bound1)

def diff_lower_bound2(model, i):
    return model.diff[i] >= model.away[i] - model.home[i]
model.diff_lb2 = Constraint(model.I, rule=diff_lower_bound2)

# objective function
model.z = Var(domain=NonNegativeReals)
model.balance_max = ConstraintList()
for i in model.I:
    model.balance_max.add(model.home[i] - model.away[i] <= model.z)
    model.balance_max.add(model.away[i] - model.home[i] <= model.z)
model.obj = Objective(expr=model.z, sense=minimize)


# solve
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False, timelimit=300)


print("result:", result)
# print("solution:")
# for w in model.W:
#     for p in model.P:
#         matrix = []
#         for i in model.I:
#             row = []
#             for j in model.J:
#                 var = model.X[w,p,i,j]
#                 if var.value is not None and abs(var.value) > 1e-6:
#                     val = int(round(value(model.X[w, p, i, j])))
#                 else:
#                     val = 0
#                 row.append(val)
#             matrix.append(row)
#         if sum(sum(i) for i in matrix) > 0:
#             print(f"\nMatrix for w={w}, p={p}:")
#             for row in matrix:
#                 print(row)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


result: 
Problem: 
- Name: unknown
  Lower bound: 1.0
  Upper bound: 1.0
  Number of objectives: 1
  Number of constraints: 110
  Number of variables: 109
  Number of nonzeros: 824
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1063
      Number of created subproblems: 1063
  Error rc: 0
  Time: 0.2158489227294922
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

solution:

Matrix for w=0, p=0:
[0, 0, 1, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]

Matrix for w=0, p=1:
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]

Matrix for w=2, p=1:
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]


In [29]:
import numpy as np

X = np.zeros((n-1, n//2, n, n))
for i in model.X.get_values().keys():
    if model.X[i].value is not None and abs(model.X[i].value) > 1e-6:
        X[i[0], i[1], i[2], i[3]] = 1
print(np.sum(X, axis=(0, 1)))

[[0. 0. 1. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 1. 0. 0.]]


Vai ai contenuti
Cronologia chat

Hai detto:
how would you convert this code for gurobipy into constraint definitions for pyomo, given the following variables:
model.W = RangeSet(0, (n-1)-1)
model.P = RangeSet(0, (n//2)-1)
model.I = RangeSet(0, n-1)
model.J = RangeSet(0, n-1)
model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)

# gurobipy code
x = m.addMVar((n-1, n//2, n, n), vtype=GRB.BINARY, name="x")

# one match against each team
m.addConstrs((
    gp.quicksum(x[i1, i2, i, i4] for i1 in range(n-1) for i2 in range(n//2) for i4 in range(n)) +
    gp.quicksum(x[i1, i2, i3, i] for i1 in range(n-1) for i2 in range(n//2) for i3 in range(n))
    == n - 1
    for i in range(n)
), name="ij")

# no match against itself
m.addConstrs((x[:, :, i, i] == 0 for i in range(n)), name="self")

# diag symmetry breaking
m.addConstrs((
    gp.quicksum(x[i1, i2, i, j] for i1 in range(n-1) for i2 in range(n//2)) +
    gp.quicksum(x[i1, i2, j, i] for i1 in range(n-1) for i2 in range(n//2))
    == 1
    for i in range(n) for j in range(n) if i != j
), name="ij")

# one match per team per week
for w in range(n-1):
    m.addConstrs((x[w, :, i, :].sum() <= 1 for i in range(n)), name="max_w")
    m.addConstrs((x[w, :, :, j].sum() <= 1 for j in range(n)), name="max_w")

# max one match over the periods in a week
for w in range(n-1):
    m.addConstrs((x[w, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_p")

# max one game per match
m.addConstrs((x[:, :, i, j].sum() <= 1 for i in range(n) for j in range(n)), name="max_ij")

# at most 2 match per period per team
m.addConstrs((
    gp.quicksum(x[w, p, i, j] for w in range(n-1) for j in range(n)) +
    gp.quicksum(x[w, p, j, i] for w in range(n-1) for j in range(n))
    <= 2
    for i in range(n) for p in range(n//2)
), name="ij")


# Objective function
h = m.addMVar((n), vtype=GRB.INTEGER, name="home")
a = m.addMVar((n), vtype=GRB.INTEGER, name="away")
d = m.addMVar((n), vtype=GRB.CONTINUOUS, name="diff")

m.addConstrs((h[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((a[i] == gp.quicksum(x[w, p, i, j] for w in range(n-1) for p in range(n//2) for j in range(n) if i != j)
              for i in range(n)))
m.addConstrs((d[i] >= h[i] - a[i] for i in range(n)))
m.addConstrs((d[i] >= a[i] - h[i] for i in range(n)))

m.setObjective(d.sum(), GRB.MINIMIZE)
ChatGPT ha detto:
Translating your Gurobi model to Pyomo involves:

Defining all variables and sets.

Converting list comprehensions (gp.quicksum) to sum(...) inside Constraint rules.

Creating Constraint objects using ConstraintList or indexed rules.

Defining the objective function using Objective.

✅ 1. Setup and Variables in Pyomo
from pyomo.environ import *

model = ConcreteModel()

# Sets
model.W = RangeSet(0, (n-1)-1)
model.P = RangeSet(0, (n//2)-1)
model.I = RangeSet(0, n-1)
model.J = RangeSet(0, n-1)

# Binary decision variable
model.X = Var(model.W, model.P, model.I, model.J, domain=Binary)

# Home, Away, Diff
model.home = Var(model.I, domain=NonNegativeIntegers)
model.away = Var(model.I, domain=NonNegativeIntegers)
model.diff = Var(model.I, domain=NonNegativeReals)
✅ 2. Constraints
a. One match against each team (sum of matches involving team i is n-1)
def one_match_rule(model, i):
    return (
        sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J) +
        sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J)
        == n - 1
    )
model.one_match_per_team = Constraint(model.I, rule=one_match_rule)
b. No matches against itself
def no_self_match_rule(model, i):
    return sum(model.X[w, p, i, i] for w in model.W for p in model.P) == 0
model.no_self_match = Constraint(model.I, rule=no_self_match_rule)
c. Diagonal symmetry breaking (i != j): Each pair plays once
def symmetry_rule(model, i, j):
    if i != j:
        return (
            sum(model.X[w, p, i, j] for w in model.W for p in model.P) +
            sum(model.X[w, p, j, i] for w in model.W for p in model.P)
            == 1
        )
    return Constraint.Skip
model.symmetry = Constraint(model.I, model.J, rule=symmetry_rule)
d. One match per team per week
def one_match_per_week_rule_i(model, w, i):
    return sum(model.X[w, p, i, j] for p in model.P for j in model.J) <= 1
model.max_one_match_i = Constraint(model.W, model.I, rule=one_match_per_week_rule_i)

def one_match_per_week_rule_j(model, w, j):
    return sum(model.X[w, p, i, j] for p in model.P for i in model.I) <= 1
model.max_one_match_j = Constraint(model.W, model.J, rule=one_match_per_week_rule_j)
e. Max one match per period in a week
def max_one_per_period_per_week(model, w, i, j):
    return sum(model.X[w, p, i, j] for p in model.P) <= 1
model.max_one_period = Constraint(model.W, model.I, model.J, rule=max_one_per_period_per_week)
f. Max one game per match over all weeks/periods
def max_one_game_per_match(model, i, j):
    return sum(model.X[w, p, i, j] for w in model.W for p in model.P) <= 1
model.max_one_match = Constraint(model.I, model.J, rule=max_one_game_per_match)
g. At most 2 matches per period per team (home + away)
def max_two_matches_per_period(model, i, p):
    return (
        sum(model.X[w, p, i, j] for w in model.W for j in model.J) +
        sum(model.X[w, p, j, i] for w in model.W for j in model.J)
        <= 2
    )
model.max_two_matches = Constraint(model.I, model.P, rule=max_two_matches_per_period)
✅ 3. Objective Function: Balance home and away
def home_games_rule(model, i):
    return model.home[i] == sum(model.X[w, p, i, j] for w in model.W for p in model.P for j in model.J if i != j)
model.home_games = Constraint(model.I, rule=home_games_rule)

def away_games_rule(model, i):
    return model.away[i] == sum(model.X[w, p, j, i] for w in model.W for p in model.P for j in model.J if i != j)
model.away_games = Constraint(model.I, rule=away_games_rule)

def diff_lower_bound1(model, i):
    return model.diff[i] >= model.home[i] - model.away[i]
model.diff_lb1 = Constraint(model.I, rule=diff_lower_bound1)

def diff_lower_bound2(model, i):
    return model.diff[i] >= model.away[i] - model.home[i]
model.diff_lb2 = Constraint(model.I, rule=diff_lower_bound2)

model.obj = Objective(expr=sum(model.diff[i] for i in model.I), sense=minimize)
✅ Summary
✅ Converted all Gurobi constraints to Pyomo.

✅ Used standard Pyomo idioms (e.g., Constraint.Skip, sum(...) for expressions).

✅ Ensured indexing and logic aligns between the models.

Let me know if you want to run or visualize a small example of this model for testing.

Hai detto:
Do you think there are ways to improve the efficiency of the code and/or reduce the number of constraints?
ChatGPT ha detto:
Yes, your current Pyomo model is a faithful and correct translation of the Gurobi version, but it's not yet optimized for efficiency. Below are several practical strategies to reduce model size, tighten the formulation, and speed up solving:

✅ Efficiency Improvements
1. Avoid Redundant Constraints
Some constraints appear to double-count or are symmetrical and can be pruned.

🔁 a. Symmetric Matches
Your symmetry constraint:

sum(X[w,p,i,j]) + sum(X[w,p,j,i]) == 1 for i != j
This includes both (i, j) and (j, i) for all i ≠ j, so it's duplicated.

✅ Optimization:
Only enforce for i < j:

def symmetry_rule(model, i, j):
    if i < j:
        return (
            sum(model.X[w, p, i, j] for w in model.W for p in model.P) +
            sum(model.X[w, p, j, i] for w in model.W for p in model.P)
            == 1
        )
    return Constraint.Skip
This halves the number of these constraints (from n(n-1) to n(n-1)/2).

2. Combine or Remove Redundant Constraints
You define both:

# one match per team per week (home side)
sum(X[w, :, i, :]) <= 1
# one match per team per week (away side)
sum(X[w, :, :, i]) <= 1
But since i appears either as home or away, and each pair plays once per week, you could combine them:

✅ Optimization:

sum(X[w, p, i, j] + X[w, p, j, i] for p in model.P for j in model.J if j != i) <= 1
This replaces two constraints with one tighter constraint per (w, i).

3. Limit Domain of Variables
You define:

model.X = Var(W, P, I, J, domain=Binary)
But most X[w,p,i,j] where i == j are never used (self-matches forbidden).

✅ Optimization:
Declare variables only where i ≠ j:

model.match_idx = [(w, p, i, j) for w in range(n-1) for p in range(n//2) for i in range(n) for j in range(n) if i != j]
model.X = Var(model.match_idx, domain=Binary)
This reduces the number of variables by ~1/n.

4. Use Implicit Constraints from Domain Logic
You currently enforce:

sum(X[..., i, i]) == 0
But if you don't define X[w, p, i, i] at all (see previous tip), this constraint is unnecessary.

✅ Optimization: Remove no_self_match entirely if variable domain excludes i == j.

5. Vectorized or Aggregated Expressions
For large problems, you can precompute expressions (e.g., using Expression objects) and reuse them instead of repeating sum(...) over same indexes in multiple constraints and objective.

6. Tighten the Objective
The current objective is:

minimize sum(diff[i])
But diff[i] >= |home[i] - away[i]|, so diff[i] can be large. Depending on the goal, minimizing the maximum difference across teams may be tighter:

✅ Alternative: Minimize max imbalance

model.z = Var(domain=NonNegativeReals)
model.balance_max = ConstraintList()
for i in model.I:
    model.balance_max.add(model.home[i] - model.away[i] <= model.z)
    model.balance_max.add(model.away[i] - model.home[i] <= model.z)
model.obj = Objective(expr=model.z, sense=minimize)
This might guide the solver faster toward balanced solutions, depending on the application.

📊 Summary of Gains
Technique	Benefit
Remove i == j from X	Reduces vars by ~1/n
Half symmetry constraints	Reduces constraints by 50%
Combine weekly match constraints	Simplifies model
Use tighter max objective	Better solver performance
Avoid redundant constraints	Smaller model footprint
🧪 Want Help Testing?
I can help you benchmark the original vs optimized model using a test case (say n=6), or generate compact data files for you to debug efficiently. Let me know!

Hai detto:
are there more efficient definitions of variables possible? like employing multiple arrays or other structures
ChatGPT ha detto:
Yes — you can significantly improve efficiency of your Pyomo model by restructuring how your decision variables are defined and represented. Instead of using a single large 4D binary variable array, you can:

🔄 Redefine Variables for Efficiency
The current variable:

model.X[w, p, i, j]  # Binary variable for week w, period p, team i vs team j
is dense and high-dimensional — for n=10, this is (9 x 5 x 10 x 10) = 4500 variables!

Many of these variables are unused or constrained to be 0 (e.g., self-matches i==j), or redundant due to symmetry.

✅ Option 1: Use a Structured Match Set
Instead of all (i, j) pairs, define only unordered match pairs (i < j), and represent the match assignment with a single binary variable per (w, p, i, j) where i < j:

model.Matches = Set(dimen=2, initialize=[(i, j) for i in range(n) for j in range(i+1, n)])
model.X = Var(model.W, model.P, model.Matches, domain=Binary)
This reduces the number of (i, j) match pairs from n² to n(n-1)/2.

You then interpret:

X[w, p, (i, j)] = 1 → match between i and j is scheduled in week w, period p.

Home/away must be tracked separately.

✅ Option 2: Split Variables for Home/Away
If you want to explicitly track home vs. away games, split X into two 3D variables:

model.X_home = Var(model.W, model.P, model.Matches, domain=Binary)
model.X_away = Var(model.W, model.P, model.Matches, domain=Binary)
Then constrain that only one of the two can be active (to represent which team is home):

def home_away_unique(model, w, p, i, j):
    return model.X_home[w, p, (i,j)] + model.X_away[w, p, (i,j)] <= 1
model.home_away_exclusive = Constraint(model.W, model.P, model.Matches, rule=home_away_unique)
This structure allows:

Cleaner modeling of balance (home vs away),

Easier symmetry breaking,

Fewer variables than full (i,j) combinations,

More readable constraint logic.

✅ Option 3: Model Matches per Week Instead of Periods
If you assume each week has a fixed number of matches (say n/2), you can model match slots per week, and assign teams to those:

model.Slots = RangeSet(0, n//2 - 1)
model.Match = Var(model.W, model.Slots, model.Matches, domain=Binary)
Then add constraints that:

Each team appears at most once per week.

Only one match per slot per week.

This structure helps:

Eliminate the period dimension if you don't care about period ordering,

Focus on weekly assignment instead of overdefining the schedule.

✅ Option 4: Use an Assignment-Based Variable
Define:

model.Play = Var(model.W, model.Slots, model.I, domain=Binary)
This says: team i plays in week w, slot s. Then define:

model.Opponent = Var(model.W, model.Slots, model.I, domain=Integers, bounds=(0, n-1))
You then:

Track who team i plays in that slot.

Encode symmetric pairings and logic with fewer binary variables.

⚠️ But: this approach is more complex and works best with logic-heavy solvers (e.g., CP).

🔍 Tradeoff Summary
Variable Definition	Complexity	Variable Count	Best For
4D X[w,p,i,j]	Simple	High	Rapid prototyping
X[w,p,(i,j)] with i<j	Medium	Medium	Symmetric schedules
Home/Away split	Medium	Medium	Balance analysis
Assignment + Opponent	High	Low	Constraint Programming / logic-heavy problems
🔧 Recommendation for Pyomo + MILP
For a performance-focused Pyomo MILP model:

✅ Use Matches = {(i, j): i < j} as the core set.

✅ Use a single variable X[w, p, (i,j)].

✅ Add a single is_home[w, p, (i,j)] binary variable if you want to track home/away.

✅ Skip all X[w,p,i,j] where i == j or (i > j).

Would you like me to refactor your Pyomo model using this optimized structure with match sets and home/away tracking?




Nessun file selezionatoNessun file selezionato
ChatGPT può commettere errori. Assicurati di verificare le informazioni importanti. Vedi Preferenze sui cookie.
